In [1]:
import pandas as pd
import re
import string

In [2]:
pd.set_option('display.max_colwidth', -1)

In [3]:
train = pd.read_csv("train.csv",encoding = "ISO-8859-1")
train.head(5)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1


In [4]:
test = pd.read_csv("test.csv",encoding = "ISO-8859-1")
test.head(5)

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, stay safe everyone."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all"
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


## Cuento los links de cada tweet: (tengo que hacerlo antes de limpiarlos porque en la limpieza se pierden)

In [5]:
train['cantidad_links'] = train['text'].str.count('http')

In [6]:
test['cantidad_links'] = test['text'].str.count('http')

In [7]:
train.nlargest(30,'cantidad_links').head()

,id,keyword,location,text,target,cantidad_links
40,59,ablaze,Live On Webcam,Check these out: http://t.co/rOI2NSmEJJ http://t.co/3Tj8ZjiN21 http://t.co/YDUiXEfIpE http://t.co/LxTjc87KLS #nsfw,0,4
48,68,ablaze,Live On Webcam,Check these out: http://t.co/rOI2NSmEJJ http://t.co/3Tj8ZjiN21 http://t.co/YDUiXEfIpE http://t.co/LxTjc87KLS #nsfw,0,4
7082,10143,upheaval,Attock,Ancient Mayan Tablet found via http://t.co/LmUMzkLtln http://t.co/yebxxAryBF http://t.co/SRRUqfffr6 http://t.co/CadzxAgMSI,0,4
1060,1531,bomb,NaN,beforeitsnews : Global Derivatives: $1.5 Quadrillion Time Bomb http://t.co/GhmmUj7GbE (vÂÃ_ http://t.co/u9LvvLzhYe) http://t.co/LyJ57pq3yX,0,3
1373,1980,bush%20fires,Trinidad and Tobago,Drought fuels bush fires in Jamaica - http://t.co/ZDtDqQbAHC http://t.co/PsQCNsVfgP - @JamaicaObserver @cnewslive RE https://t.co/6ZGef8J8Bm,1,3


## Signos de pregunta

In [8]:
test['cantidad_signos_pregunta'] = test['text'].str.count('\?')

In [9]:
train['cantidad_signos_pregunta'] = train['text'].str.count('\?')

## Limpieza de los textos

In [10]:
train_limpio = train.copy()

In [11]:
test_limpio = test.copy()

In [12]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()  #quito_mayusculas
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  #quito caracteres inusuales
    text = re.sub('â' , '', text)  #quito caracteres inusuales
    text = re.sub('ª' , '', text)  #quito caracteres inusuales
    text = re.sub('ã' , '', text)  #quito caracteres inusuales
    text = re.sub('¼' , '', text)  #quito caracteres inusuales
    text = re.sub('\w*\d\w*', '', text)  # quito numeros
    return text

In [13]:
train_limpio['text'] = train_limpio['text'].apply(lambda x: clean_text(x))
train_limpio.head()

,id,keyword,location,text,target,cantidad_links,cantidad_signos_pregunta
0,1,NaN,NaN,our deeds are the reason of this earthquake may allah forgive us all,1,0,0
1,4,NaN,NaN,forest fire near la ronge sask canada,1,0,0
2,5,NaN,NaN,all residents asked to shelter in place are being notified by officers no other evacuation or shelter in place orders are expected,1,0,0
3,6,NaN,NaN,people receive wildfires evacuation orders in california,1,0,0
4,7,NaN,NaN,just got sent this photo from ruby alaska as smoke from wildfires pours into a school,1,0,0


In [14]:
test_limpio['text'] = test_limpio['text'].apply(lambda x: clean_text(x))

In [15]:
train_limpio.columns

Index(['id', 'keyword', 'location', 'text', 'target', 'cantidad_links',
       'cantidad_signos_pregunta'],
      dtype='object')

## Analizar las repeticiones de un tweet

In [16]:
train_limpio['repeticiones'] = train_limpio.groupby('text')['id'].transform('count')
train_limpio.head()

,id,keyword,location,text,target,cantidad_links,cantidad_signos_pregunta,repeticiones
0,1,NaN,NaN,our deeds are the reason of this earthquake may allah forgive us all,1,0,0,1
1,4,NaN,NaN,forest fire near la ronge sask canada,1,0,0,1
2,5,NaN,NaN,all residents asked to shelter in place are being notified by officers no other evacuation or shelter in place orders are expected,1,0,0,1
3,6,NaN,NaN,people receive wildfires evacuation orders in california,1,0,0,1
4,7,NaN,NaN,just got sent this photo from ruby alaska as smoke from wildfires pours into a school,1,0,0,1


In [17]:
train_limpio.nlargest(20,'repeticiones').head()

,id,keyword,location,text,target,cantidad_links,cantidad_signos_pregunta,repeticiones
5876,8393,sandstorm,USA,watch this airport get swallowed up by a sandstorm in under a minute,1,1,0,22
5877,8394,sandstorm,NaN,watch this airport get swallowed up by a sandstorm in under a minute,1,1,0,22
5879,8397,sandstorm,NaN,watch this airport get swallowed up by a sandstorm in under a minute,1,1,0,22
5882,8401,sandstorm,hkXfYMhEx,watch this airport get swallowed up by a sandstorm in under a minute,1,1,0,22
5883,8402,sandstorm,US,watch this airport get swallowed up by a sandstorm in under a minute,1,1,0,22


In [18]:
test_limpio['repeticiones'] = test_limpio.groupby('text')['id'].transform('count')

In [19]:
#Remuevo informacion
##Remuevo el ID, y remuevo keyword y location, por ahora

In [20]:
train_limpio = train_limpio.drop(columns=['id'])
train_limpio = train_limpio.drop(columns=['keyword','location'])

## Remuevo los datos repetidos 
### No remuevo los repetivos del test obviamente

In [21]:
train_limpio.shape

(7613, 5)

In [22]:
train_limpio = train_limpio.drop_duplicates()
train_limpio.shape

(7134, 5)

In [23]:
train_limpio

,text,target,cantidad_links,cantidad_signos_pregunta,repeticiones
0,our deeds are the reason of this earthquake may allah forgive us all,1,0,0,1
1,forest fire near la ronge sask canada,1,0,0,1
2,all residents asked to shelter in place are being notified by officers no other evacuation or shelter in place orders are expected,1,0,0,1
3,people receive wildfires evacuation orders in california,1,0,0,1
4,just got sent this photo from ruby alaska as smoke from wildfires pours into a school,1,0,0,1
5,rockyfire update california hwy closed in both directions due to lake county fire cafire wildfires,1,0,0,1
6,flood disaster heavy rain causes flash flooding of streets in manitou colorado springs areas,1,0,0,1
7,im on top of the hill and i can see a fire in the woods,1,0,0,1
8,theres an emergency evacuation happening now in the building across the street,1,0,0,1
9,im afraid that the tornado is coming to our area,1,0,0,1


# Longitud de los tweets

In [24]:
import math

def roundup(x):
    return int(math.ceil(x / 10.0)) * 10

In [25]:
train_limpio['longitud_tweet'] = train_limpio['text'].apply(lambda x: roundup(len(x)))

In [26]:
train_limpio.head(2)

,text,target,cantidad_links,cantidad_signos_pregunta,repeticiones,longitud_tweet
0,our deeds are the reason of this earthquake may allah forgive us all,1,0,0,1,70
1,forest fire near la ronge sask canada,1,0,0,1,40


In [27]:
test_limpio['longitud_tweet'] = test_limpio['text'].apply(lambda x: roundup(len(x)))

## Caso abarcativo: agrupo por palabras y me quedon con las mas repetidas

In [28]:
import re 
from string import punctuation

palabras_por_separado = train_limpio.copy()
palabras_por_separado.set_index(['target'],inplace=True)
palabras_por_separado = palabras_por_separado['text'].str.split(expand=True).stack().to_frame()
palabras_por_separado.columns = ['word']
palabras_por_separado['word'] = palabras_por_separado['word'].str.replace(rf'[{punctuation}]', '')
palabras_por_separado.reset_index(inplace=True)
del palabras_por_separado['level_1']
palabras_mas_comunes = palabras_por_separado.groupby('word')['target'].count().rename('cantidad_apariciones').reset_index().nlargest(400,'cantidad_apariciones')
palabras_mas_comunes.head()

,word,cantidad_apariciones
15299,the,3070
0,a,2036
15570,to,1853
7907,in,1816
10982,of,1674


In [29]:
for i in palabras_mas_comunes.word.values:
    train_limpio[i]= train_limpio['text'].apply(lambda x: x.split(' ').count(i) )
    test_limpio[i]= test_limpio['text'].apply(lambda x: x.split(' ').count(i) )

## Similar al anterior: agrupo por 2-shingles y me quedo con los que mas veces aparecen 
### En shingles puede apreciarse tambien informacion valiosa

In [30]:
palabras_por_separado = train_limpio.copy()
palabras_por_separado.set_index(['target'],inplace=True)
palabras_por_separado = palabras_por_separado['text'].str.split(expand=True).stack().to_frame()
palabras_por_separado.columns = ['word']
palabras_por_separado = palabras_por_separado.reset_index()
palabras_por_separado = palabras_por_separado[['word']]
palabras_por_separado.head()

,word
0,our
1,deeds
2,are
3,the
4,reason


In [31]:
palabras_shifteadas = train_limpio.copy()
palabras_shifteadas.set_index(['target'],inplace=True)
palabras_shifteadas = palabras_shifteadas['text'].str.split(expand=True).stack().to_frame()
palabras_shifteadas.columns = ['next_word']
palabras_shifteadas = palabras_shifteadas.reset_index()
palabras_shifteadas = palabras_shifteadas[['next_word']]
palabras_shifteadas = palabras_shifteadas.shift(-1)
palabras_shifteadas.head()

,next_word
0,deeds
1,are
2,the
3,reason
4,of


In [32]:
palabras_shifteadas.shape

(97878, 1)

In [33]:
dos_shingles = palabras_por_separado.merge(palabras_shifteadas, left_index=True, right_index=True)
dos_shingles.head()

,word,next_word
0,our,deeds
1,deeds,are
2,are,the
3,the,reason
4,reason,of


In [34]:
dos_shingles['shingle'] = dos_shingles['word']+' '+dos_shingles['next_word']
dos_shingles['valor'] = 1
dos_shingles.head()

,word,next_word,shingle,valor
0,our,deeds,our deeds,1
1,deeds,are,deeds are,1
2,are,the,are the,1
3,the,reason,the reason,1
4,reason,of,reason of,1


In [35]:
dos_shingles_agrupados = dos_shingles.groupby('shingle').valor.sum().rename('cantidad_apariciones').to_frame()
shingles_mas_comunes = dos_shingles_agrupados.nlargest(400,'cantidad_apariciones')
shingles_mas_comunes.head()

,cantidad_apariciones
shingle,
in the,291
of the,234
on the,128
to the,120
in a,108


In [36]:
for i in shingles_mas_comunes.index:
    train_limpio[i]= train_limpio['text'].apply(lambda x: x.count(i) )
    test_limpio[i]= test_limpio['text'].apply(lambda x: x.count(i) )

# Remocion de StopWords
### Lo deje comentado porque era demasiado lento

In [37]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
len(stop)

[nltk_data] Downloading package stopwords to /home/andres/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


179

In [38]:
#stop

In [39]:
#train_limpio['text']=train_limpio['text'].apply(lambda x: [item for item in x.split() if item not in stop])
#train_limpio.head()

In [40]:
# Como volver a convertirlo en texto (no lo voy a hacer por ahora)
# train_limpio['text'] = train_limpio['text'].apply(lambda x: ' '.join(x))
# train_limpio.head()

# Correccion de typos (me conviene correjirlos o simplemente sacarlos y agregar en otra columna "cant palabras mal escritas"?)

### Lo deje comentado por ser demasiado lento

## Opcion 1: las corrijo

In [41]:
#pruebita = train_limpio.iloc[1:150].copy()

In [42]:
#pruebita

In [43]:
#pruebita['text'] = pruebita['text'].apply(lambda x: TextBlob(x).correct() )
#pruebita.text = pruebita.text.apply(lambda txt: ''.join(TextBlob(txt).correct()))

###  Muuy lento, como 3 minutos en corregir typos de apenas 150 filas

# Opcion 2: contar cuantas palabras mal escritas tiene cada texto

In [44]:
#from nltk.corpus import brown
#nltk.download('brown')
#word_list = brown.words()
#word_set = set(word_list)

In [45]:
#len(word_list)

In [46]:
def cant_misspelled(x):
    misspelled_words = 0
    for i in x:
        if (i not in word_list):
            misspelled_words=misspelled_words+1
    return(misspelled_words)

In [47]:
x=['hello', 'there', 'general', 'nto']
x

['hello', 'there', 'general', 'nto']

In [48]:
#cant_misspelled(x)

### Mas lento todavia...  pruebo otra cosa

## Contar la cantidad de veces seguidas que se repite una letra, si alguna se repite mas de 3 veces, es probable que haya sido intencional, lo cual le quita credibildad

In [49]:
train_limpio.head(2)

,text,target,cantidad_links,cantidad_signos_pregunta,repeticiones,longitud_tweet,the,a,to,in,...,on your,out with,outrage amid,over the,patience jonathan,plans to,police post,provoke outrage,read by,rescuers are
0,our deeds are the reason of this earthquake may allah forgive us all,1,0,0,1,70,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,forest fire near la ronge sask canada,1,0,0,1,40,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
def letras_repetidas(text):
    max_repeteated_letters = 0
    current_repeated_letters = 1
    for word in text:
        if(current_repeated_letters>max_repeteated_letters):
            max_repeteated_letters=current_repeated_letters
        current_repeated_letters=1
        for i in range (len(word)-1):
            if(word[i]==word[i+1]):
                current_repeated_letters=current_repeated_letters+1
            else:
                if(current_repeated_letters>max_repeteated_letters):
                    max_repeteated_letters=current_repeated_letters
                current_repeated_letters=1
    return (max_repeteated_letters)

In [51]:
def remover_palabras_con_letras_repetidas(text):
    max_repeated_letters = 0
    current_repeated_letters = 1
    new_text = []
    for word in text:
        current_repeated_letters=1
        max_repeated_letters = 1
        for i in range (len(word)-1):
            if(word[i]==word[i+1]):
                current_repeated_letters=current_repeated_letters+1
            else:
                if(current_repeated_letters>max_repeated_letters):
                    max_repeated_letters=current_repeated_letters
                current_repeated_letters=1
        if(current_repeated_letters<=2 and max_repeated_letters<=2):
            new_text.append(word)  #remuevo las palabras que tengan 3 o mas letras seguidas repetidas
    return (new_text)

In [52]:
train_limpio['letras_seguidas'] = train_limpio['text'].apply(lambda x: letras_repetidas(x.split(' ')))

In [53]:
test_limpio['letras_seguidas'] = test_limpio['text'].apply(lambda x: letras_repetidas(x.split(' ')))

In [54]:
#Remuevo esas palabras
train_limpio['text'] = train_limpio['text'].apply(lambda x: remover_palabras_con_letras_repetidas(x.split(' ')))
test_limpio['text'] = test_limpio['text'].apply(lambda x: remover_palabras_con_letras_repetidas(x.split(' ')))
train_limpio.head(2)

,text,target,cantidad_links,cantidad_signos_pregunta,repeticiones,longitud_tweet,the,a,to,in,...,out with,outrage amid,over the,patience jonathan,plans to,police post,provoke outrage,read by,rescuers are,letras_seguidas
0,"[our, deeds, are, the, reason, of, this, earthquake, may, allah, forgive, us, all]",1,0,0,1,70,1,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1,"[forest, fire, near, la, ronge, sask, canada]",1,0,0,1,40,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [55]:
len(train_limpio[train_limpio['letras_seguidas']>2])

135

In [56]:
train_limpio[train_limpio['letras_seguidas']>2].target.mean()

0.37037037037037035

## Parece dar buenos resultados pero obviamente no reemplaza un programa que me detecte cualquier error ortografico

## Columna si contiene insultos, o la cantidad que tiene

In [57]:
#Estoy usando el train original porque para este prueba simple uso los textos aun no separados por palabra
insults_list = ['idiot','fuck','moron','scum','bitch','shit','bastard','cunt','ridicul','tramp']
mean_insult = [train[train['text'].str.contains(i)].target.mean() for i in insults_list]
times_insult_used = [len(train[train['text'].str.contains(i)]) for i in insults_list]

In [58]:
print(insults_list)
print(times_insult_used)
print(mean_insult)

['idiot', 'fuck', 'moron', 'scum', 'bitch', 'shit', 'bastard', 'cunt', 'ridicul', 'tramp']
[4, 67, 1, 1, 10, 57, 1, 4, 4, 1]
[0.5, 0.208955223880597, 0.0, 0.0, 0.0, 0.3157894736842105, 0.0, 0.0, 0.25, 1.0]


In [59]:
def insults(text):
    insults = ['idiot','fuck','moron','scum','bitch','shit','bastard','cunt','ridicul','tramp']
    amount = 0
    for word in text:
        if(word in insults):
            amount = amount +1
    return(amount)

In [60]:
train_limpio['insultos_por_tweet'] = train_limpio['text'].apply(lambda x: insults(x) )
test_limpio['insultos_por_tweet'] = test_limpio['text'].apply(lambda x: insults(x) )

In [61]:
train_limpio[train_limpio['insultos_por_tweet']==1].target.mean()

0.25773195876288657

In [62]:
# Si bien algunos aparecen mucho menos que otros, en todos se cumple que estan por debajo del 0.5

In [63]:
# Remuevo estas palabras de los textos
# Al final no las remuevo, por que lo haria?
#train_limpio['text']=train_limpio['text'].apply(lambda x: [item for item in x if item not in insults_list])

## Remover expresiones

In [64]:
#Estoy usando el train original porque para este prueba simple uso los textos aun no separados por palabra
expressions = ['lol','omg','wtf','lmao','imo']  #agregar mas,no importa que el train no tenga, si el test si,joya
mean_expressions = [train[train['text'].str.contains(i)].target.mean() for i in expressions]
times_expressions_used = [len(train[train['text'].str.contains(i)]) for i in expressions]

In [65]:
print(expressions)
print(times_expressions_used)
print(mean_expressions)

['lol', 'omg', 'wtf', 'lmao', 'imo']
[54, 6, 8, 14, 15]
[0.12962962962962962, 0.3333333333333333, 0.375, 0.0, 0.3333333333333333]


In [66]:
def expressions_counter(text):
    expressions = ['lol','omg','wtf','lmao','imo']
    amount = 0
    for word in text:
        if(word in expressions):
            amount = amount +1
    return(amount)

In [67]:
train_limpio['expresiones_por_tweet'] = train_limpio['text'].apply(lambda x: expressions_counter(x) )
test_limpio['expresiones_por_tweet'] = test_limpio['text'].apply(lambda x: expressions_counter(x) )

In [68]:
# Remuevo estas palabras de los textos
# Al final no las remuevo, por que lo haria?
#train_limpio['text']=train_limpio['text'].apply(lambda x: [item for item in x if item not in expressions])

## Cuento pronombres personales,esto se habia hecho en el TP1 y dio buenos resultados

In [69]:
#Estoy usando el train original porque para este prueba simple uso los textos aun no separados por palabra
personal_pronouns = ['i','me','you','she','her','him','he','his','we','us','them','they','im']  #agregar mas,no importa que el train no tenga, si el test si,joya
mean_personal_pronouns = [train[train['text'].str.contains(i)].target.mean() for i in personal_pronouns]
times_personal_pronoun_used = [len(train[train['text'].str.contains(i)]) for i in personal_pronouns]

In [70]:
print(personal_pronouns)
print(mean_personal_pronouns)
print(times_personal_pronoun_used)

['i', 'me', 'you', 'she', 'her', 'him', 'he', 'his', 'we', 'us', 'them', 'they', 'im']
[0.4389975213439824, 0.43219954648526077, 0.2056921086675291, 0.4017857142857143, 0.42958579881656805, 0.7189189189189189, 0.4109311740890688, 0.37099494097807756, 0.4495515695067265, 0.4447797223898612, 0.3557692307692308, 0.3006134969325153, 0.5167630057803468]
[7262, 2205, 773, 224, 845, 185, 3952, 593, 892, 1657, 104, 163, 865]


In [71]:
for i in personal_pronouns:
    train_limpio[i]= train_limpio['text'].apply(lambda x: x.count(i) )
    test_limpio[i]= test_limpio['text'].apply(lambda x: x.count(i) )

## Palabras mas comunes (esta parte no se si deberia quedar así basandome en las del TP1 o rehacer eso ahora que está mas limpio el train o hacerlo de otra forma)

In [72]:
palabras = ['hiroshima','bombing','kill','california','suicide','fire','forest','train','accident','disaster'\
                ,'new','crash','storm','police','nuclear','two','fire','dead','died','years']
for i in palabras:
    train_limpio[i]= train_limpio['text'].apply(lambda x: x.count(i) )
    test_limpio[i]= test_limpio['text'].apply(lambda x: x.count(i) )

In [73]:
#muchas de estas palabras ya estaban agregadas por otros medios

## Guardo los csvs limpios para mas adelante

In [74]:
train_limpio.to_csv('train_limpio.csv', index=False)

In [75]:
test_limpio.to_csv('test_limpio.csv', index=False)

## Redes neuronales con tensorFlow (lo pongo aca xq no consegui que ande)

In [76]:
#import tensorflow as tf
#tf.__version__

In [77]:
#mnist = tf.keras.datasets.mnist
#(x_train, y_train) , (x_test, y_test) = mnist.load_data()

In [78]:
#import matplotlib.pyplot as plt
#plt.imshow(x_train[0])
#plt.show()

In [79]:
#normalizo la data para reder neuronales para que prediga mejor
#x_train = tf.keras.utils.normalize(x_train, axis=1)
#x_test = tf.keras.utils.normalize(x_test, axis=1)

In [80]:
# armo el modelo

## Vectorizo los textos (quiza la parte mas importante de todo el cleaning y apenas lo toque)

In [81]:
train_solo_texto = train_limpio[['id','text']].copy()

KeyError: "['id'] not in index"

In [ ]:
train_solo_texto['text'] = train_solo_texto['text'].apply(lambda x: ' '.join(x))

In [ ]:
train_solo_texto.head()

In [ ]:
train_solo_texto.groupby('text')['id'].transform('count')

In [ ]:
train_solo_texto.groupby('text').id.count().to_frame().reset_index().nlargest(40,'id')

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
#agregarle n-shingles despues
cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(train_solo_texto.text)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = train_solo_texto.index
data_dtm

In [ ]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
stemmed = [porter.stem(word) for word in tokens]
#train_solo_texto['stemmed'] = train_solo_texto['text'].apply(lambda x: porter.stem(word) for word in x)

In [ ]:
train_solo_texto['stemmed'] = train_solo_texto['text'].apply(lambda x: porter.stem(word) for word in x)